# FrozenLake 8x8

Opis gry: https://gym.openai.com/envs/FrozenLake8x8-v0

Celem gry we FrozenLake jest przejście z punktu początkowego (S) do końcowego (G). Gracz może się poruszać po polach oznaczonych jako (F). Wejście na pole (H) kończy grę. Po wybraniu akcji istnieje pewne prawdopodobieństwo wykonania innego ruchu niż wskazany.

    "SFFFFFFF",
    "FFFFFFFF",
    "FFFHFFFF",
    "FFFFFHFF",
    "FFFHFFFF",
    "FHHFFFHF",
    "FHFFHFHF",
    "FFFHFFFG"

Zadanie:
- zaimplementować Q-learning (w wersji niedeterministycznej)
- dobrać parametry 

Opis stanu gry:
- stan gry jest reprezentowany przez jedną liczbę

Wskazówka: Dotarcie do stanu końcowego przy losowym wyborze akcji może wymagać bardzo wielu prób.

In [ ]:
try:
    import gym
except:
    !apt-get install cmake swig
    !pip install gym[all]

data_url = 'https://www.mimuw.edu.pl/~mm319369/priv/d73890416bec03ff3e2b3756af8c941c/task1-data.zip'

def download_zip(url):
    from zipfile import ZipFile
    try:
        from urllib import urlretrieve
    except:
        from urllib.request import urlretrieve
    from tempfile import mktemp

    filename = mktemp('.zip')
    name, hdrs = urlretrieve(data_url, filename)
    thefile = ZipFile(filename)
    thefile.extractall('')
    thefile.close()
    
import os

if not os.path.exists('taxi_custom.py'):
    download_zip(data_url)
    print("Done downloading")
else:
    print("Files already exist")

Done downloading


In [ ]:
import numpy as np
import gym
import random
from collections import defaultdict

ENVIRONMENT = 'FrozenLake8x8-v0'
SOLVE_SCORE = 0.8

In [ ]:
class QPlayer(object):
    def __init__(self, num_of_actions, num_of_states=None):
        #Uzupełnij wartości parametrów
        self.lr = 0.04 #learning rate: 0.0 - 1.0
        self.gamma = 0.99 #gamma (discount factor): 0.0 - 1.0
        self.epsilon_start = 1 #wartość początkowa dla epsilon-greedy: 0.0 - 1.0
        self.epsilon_min = 0.01 #wartosc koncowa dla epsilon-greedy: 0.0 - 1.0
        self.epsilon_decay = 0.999998 #szybkosc wygaszania epsilon: 0.9 - 0.999999...

        self.num_of_actions = num_of_actions
        self.num_of_states = num_of_states
        
        self.Q = defaultdict(lambda: np.zeros(self.num_of_actions))
    
    def encode_state(self, state):
        return state
    
    def train(self, env, steps):
        # Metoda 'train' przyjmuje jako parametry zainicjowane srodowisko ai gym (env) oraz 
        # liczbę kroków uczenia (steps). Wewnątrz tej metody należy zaimplementować uczenie Q-learning
        # Należy wykorzystac dobrane parametry uczenia (lr, gamma,...) zdefiniowane w __init__
        
        epsilon = self.epsilon_start
        rewards = []
        counter = 0

        while steps > 0: 
          state = env.reset()
          end = False
          counter += 1
          score = 0

        
          while not end and steps > 0:

            n = np.random.random(1)[0]

            if n < epsilon:
              action = env.action_space.sample() 
            else:
              action = np.argmax(self.Q[state]) 

            new_state, reward, end, _ = env.step(action)

            if end:
              self.Q[state][action] = self.lr * reward + (1 - self.lr) * self.Q[state][action]
            else:
              self.Q[state][action] += self.lr * (reward + self.gamma * np.max(self.Q[new_state]) - self.Q[state][action])


            if steps % 100000 == 0:
              if not rewards:
                self.data = np.array([counter, steps, 0, epsilon])
                print(counter, steps, "\t", epsilon, sep='\t')
              else:
                print(counter, steps, np.mean(rewards), epsilon, sep='\t')
                self.data = np.append(self.data, [counter, steps, np.mean(rewards), epsilon])
                rewards = []

           
            state = new_state
            steps -= 1;
            score += reward
            if epsilon > self.epsilon_min:
                epsilon *= self.epsilon_decay
            
        rewards.append(score)

    
    def play(self, state):
        # Ta metoda zwraca (wyuczoną) akcje dla danego stanu
        # Uzywana do 'grania' już wytrenowanym agentem
        s = self.encode_state(state)
        return np.argmax(self.Q[s])
        
        

In [ ]:
env = gym.make(ENVIRONMENT)
print("OBSERVATION SPACE:", env.observation_space)
print("ACTION SPACE:", env.action_space)


OBSERVATION SPACE: Discrete(64)
ACTION SPACE: Discrete(4)


In [ ]:
player = QPlayer(num_of_actions=env.action_space.n, num_of_states=env.observation_space.n)

#Trenujemy
iterations = 3500000
player.train(env, iterations)

1	3500000			1
2937	3400000			0.8187305893359925
5329	3300000			0.6703197779144778
7470	3200000			0.5488113068155073
9310	3100000			0.44932860466333335
10986	3000000			0.3678790733015422
12580	2900000			0.30119385048855674
14027	2800000			0.24659661871488162
15439	2700000			0.2018961949687037
16787	2600000			0.1652985906914241
18059	2500000			0.1353350125732031
19330	2400000			0.11080291460185661
20595	2300000			0.09071773557212548
21726	2200000			0.0742733851081955
22969	2100000			0.0608098923616115
24130	2000000			0.049786919010681524
25296	1900000			0.04076207354283997
26435	1800000			0.03337315649428631
27537	1700000			0.02732362408457046
28780	1600000			0.022370686849556468
29955	1500000			0.018315565628188663
31074	1400000			0.014995513840789482
32138	1300000			0.012277285884265741
33304	1200000			0.010051789507471586
34508	1100000			0.009999995809393139
35675	1000000			0.009999995809393139
36834	900000			0.009999995809393139
38019	800000			0.009999995809393139
39076	700000			0.00

In [ ]:
def evaluate_player(player, env):
     
    scores = []
    for i in range(10000):
        score = 0
        s = env.reset()
        while(True):
            a = player.play(s)
            s, r, end, _ = env.step(a)
            score += r
            if end:
                break
                
        #print(i, score)
        scores.append(score)
    
    return np.mean(scores)

#Testowanie
new_env = gym.make(ENVIRONMENT)
test_player = player

score = evaluate_player(test_player, new_env)
print("WYNIK:", score)

assert score >= SOLVE_SCORE, "Trzeba jeszcze popracować..."
print("GRA ROZWIĄZANA!")

WYNIK: 0.852
GRA ROZWIĄZANA!
